<a href="https://colab.research.google.com/github/luimui/AI3-NN-SS24/blob/main/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# https://github.com/karpathy/ng-video-lecture
# https://www.youtube.com/watch?v=kCc8FmEb1nY

import torch
import torch.nn as nn
from torch.nn import functional as F

import requests

In [26]:
response = requests.get('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt', allow_redirects=True)

text = response.text


# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'vocab_size: {vocab_size}')
print(''.join(chars))

vocab_size: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [27]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(f'encode - hi there: {encode("hi there")}')
print(f'decode - {encode("hi there")}: {decode(encode("hi there"))}')

encode - hi there: [46, 47, 1, 58, 46, 43, 56, 43]
decode - [46, 47, 1, 58, 46, 43, 56, 43]: hi there


In [31]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)

n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [37]:
block_size = 8
print(train_data[:block_size + 1])


x = train_data[:block_size]
y = train_data[1:block_size + 1]
for i in range(block_size):
  context = x[:i + 1]
  target = y[i]
  print(f'when input is {context} ther target is: {target}')

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
when input is tensor([18]) ther target is: 47
when input is tensor([18, 47]) ther target is: 56
when input is tensor([18, 47, 56]) ther target is: 57
when input is tensor([18, 47, 56, 57]) ther target is: 58
when input is tensor([18, 47, 56, 57, 58]) ther target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) ther target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) ther target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) ther target is: 58


In [47]:
torch.manual_seed(1337)
batch_size = 4 #parallel processing
block_size = 8 #max length of context


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    #random 4 numbers [0,datasize-block_size]
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #input, pick block_sized pieces, starting at random pos in datatset
    x = torch.stack([data[i:i+block_size] for i in ix])
    #labels for these pieces
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    #x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print(f'inputs: {xb.shape} \n {xb}')
print(f'targets: {yb.shape} \n {yb}')

inputs: torch.Size([4, 8]) 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: torch.Size([4, 8]) 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [58]:
class BigramLanguageModel(nn.Module):

  def __init__(self, covab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    #idx and targets are (B,T) tensor of integer
    logits = self.token_embedding_table(idx) #(Batch=4,Time=8,Channel=vocab_size)

    if targets is None:
      loss = None

    else:

      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets= targets.view(B*T)

      loss = F.cross_entropy(logits, targets)

    return logits, loss


  def generate(self, idx, max_new_tokens):
      # idx is (B, T) array of indices in the current context
      for _ in range(max_new_tokens):
          # get the predictions
          logits, loss = self(idx)
          # focus only on the last time step
          logits = logits[:, -1, :] # becomes (B, C)
          # apply softmax to get probabilities
          probs = F.softmax(logits, dim=-1) # (B, C)
          # sample from the distribution
          idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
          # append sampled index to the running sequence
          idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
      return idx




m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.6208, grad_fn=<NllLossBackward0>)


In [59]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens = 100)[0].tolist()))


 gtFeYzB;YrsAAZH!Pga,aZflmP'FDv'hOVU$kEBYaI-mTyiP?xL!.$U.PyL;nhM !!bQaCZUdJ3ZZH&nQzj
CZlkVSdDb
o?HJO


In [61]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [69]:
batch_size = 32
for steps in range(10000):

  #sample a batch of data
  xb, yb = get_batch('train')

  # eval loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.3433406352996826


In [71]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens = 500)[0].tolist()))


Bolowh
Treayousth, wiscous yen.

Whatinee'styoimoungo
BUERK: bur aintrougsad met INoun'ser Cacow co: twaks'r hississtou' yel llind gr ok s; sthats Thears tuchor ofasthis sthis INTher sth; s IENTASe, frepre mef chede'sthat the me nite beye g, t re GLAnder m stheo u the the fofathendre t thoo brseadassouss r tere; w


ADWhese setriesig,
APRond ocere iobllescer mile is bey, magher whetat wid;

ARINIUCICE y f weat d itit pe thareily stishay f a da aral.
Trer. saisevithesthat An d l?
Angrur wh reavil
